In [1]:
import os
print(os.getcwd())

g:\DIYguru\Data-Science-and-Engineering-Analytics


In [ ]:
'''
Created a function 'load_data_excel' to load excel into python
'''

import pandas as pd
import numpy as np
import os

def load_data_excel(file_path):
    """
    Load data from an Excel file and return a DataFrame.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    df = pd.read_excel(file_path)
    return df

In [2]:
'''
Created a function 'load csv' to load excel into python
'''

import pandas as pd
import numpy as np
import os

def load_data_csv(file_path):
    """
    Load data from an csv file and return a DataFrame.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    df = pd.read_csv(file_path)
    return df

In [ ]:
'''
Loaded VED_Static_Data_ICE&HEV into dataframe df_ICE_HEV using the above function
Loaded VED_Static_Data_PHEV&EV into dataframe df_PHEV_EV using the above function
'''

DATA_DIR = load_data_excel("G:\\DIYguru\\Notes and Sample Data\\VED-master\\Data\\VED_Static_Data_ICE&HEV.xlsx")



In [3]:
'''
Loads EV charging station datasets from CSV files for two different stations using the custom load_data_csv function.
This prepares the data for further analysis and processing in subsequent steps.
'''

# https://www.kaggle.com/datasets/datasetengineer/ev-charging-station-data-california-region?utm_source=chatgpt.com

#df1 = load_data_csv(r"G:\DIYguru\Notes and Sample Data\EV Charging Station Data- California Region\Charging station_A_Calif.csv")
df1 = load_data_csv(r"G:\DIYguru\Notes and Sample Data\EV Charging Station Data- California Region\Charging station_B__Calif.csv")
df2 = load_data_csv(r"G:\DIYguru\Notes and Sample Data\EV Charging Station Data- California Region\Charging station_C__Calif.csv")

In [4]:
'''
Concatenates the two dataframes, df1 and df2, into a single dataframe df.
This operation combines the datasets from two different EV charging stations into one unified dataset for further analysis.
The index is reset to ensure a continuous sequence after concatenation.
'''

# concatenate the dataframes
df = pd.concat([df1, df2], ignore_index=True)

In [9]:
import os
import pandas as pd

def load_files_from_directory(directory_path, file_types=('csv', 'xls', 'xlsx')):
    """
    Load and append CSV and Excel files from a given directory into a single pandas DataFrame.

    Args:
        directory_path (str): Path to the directory containing the files.
        file_types (tuple): File types to load. Default is ('csv', 'xls', 'xlsx').

    Returns:
        pd.DataFrame: A single DataFrame containing data from all files.
    """
    dataframes = []
    for filename in os.listdir(directory_path):
        filepath = os.path.join(directory_path, filename)
        if os.path.isfile(filepath):
            ext = filename.lower().split('.')[-1]
            try:
                if ext == 'csv' and 'csv' in file_types:
                    df = pd.read_csv(filepath)
                    dataframes.append(df)
                elif ext in ['xls', 'xlsx'] and ext in file_types:
                    df = pd.read_excel(filepath)
                    dataframes.append(df)
            except Exception as e:
                print(f"Failed to load {filename}: {e}")
    if dataframes:
        return pd.concat(dataframes, ignore_index=True)
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no files are loaded

In [ ]:
DATA_DIR = r"G:\DIYguru\Notes and Sample Data\EV Charging Station Data- California Region"
DATA_DIR = load_files_from_directory(DATA_DIR)

print(f"Combined DataFrame shape: {DATA_DIR.shape}")

In [5]:
'''
This cell loads AWS credentials from a .env file using python-dotenv,
creates a boto3 S3 client with those credentials,
and lists all S3 buckets in the account.
'''

import os
from dotenv import load_dotenv
import boto3

# Load environment variables from .env
load_dotenv()

# Create boto3 client using loaded environment variables
s3 = boto3.client("s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_DEFAULT_REGION")
)

# Example: list buckets
buckets = s3.list_buckets()
print("Your S3 Buckets:")
for bucket in buckets['Buckets']:
    print(f" - {bucket['Name']}")

Your S3 Buckets:
 - s3aravindh973515031797


In [10]:
bucket_name = 's3aravindh973515031797'

response = s3.list_objects_v2(Bucket=bucket_name)
for item in response.get("Contents", []):
    print(item["Key"])

Cleaned up VED Source Data/
Cleaned up VED Source Data/df_ICE_HEV.parquet
Cleaned up VED Source Data/df_PHEV_EV.parquet
Cleaned up VED Source Data/df_VED.parquet
Cleaned up VED Source Data/df_combined.parquet
Cleaned up VED Source Data/df_dynamic_sample.parquet
Cleaned up VED Source Data/df_static.parquet
DIYguru ML Source Data/
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171101_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171108_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171115_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171122_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171129_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171206_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171213_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171220_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171227_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_180103_week.csv
DIYguru ML Source D

In [7]:
def push_df_to_s3_parquet(df, object_key):
    """
    Pushes a pandas DataFrame to a predefined S3 bucket as a Parquet file.

    Args:
        df (pd.DataFrame): DataFrame to upload.
        object_key (str): S3 object key (path/filename.parquet).
    """
    from io import BytesIO
    parquet_buffer = BytesIO()
    df.to_parquet(parquet_buffer, index=False)
    parquet_buffer.seek(0)
    s3.upload_fileobj(parquet_buffer, bucket_name, object_key)
    print(f"DataFrame uploaded to s3://{bucket_name}/{object_key}")

# Example usage:
# push_df_to_s3_parquet(df_static, 'path/to/df_static.parquet')

In [8]:
push_df_to_s3_parquet(df, 'EV Charging Station/Charging_data.parquet')

DataFrame uploaded to s3://s3aravindh973515031797/EV Charging Station/Charging_data.parquet


In [9]:
from io import BytesIO

def read_parquet_from_s3(bucket_name, object_key):
        """
        Reads a Parquet file from an AWS S3 bucket using the global s3 client.

        Args:
            bucket_name: Name of the S3 bucket.
            object_key: Key (path) to the Parquet file in the S3 bucket.

        Returns:
            DataFrame containing the Parquet data.
        """
        response = s3.get_object(Bucket=bucket_name, Key=object_key)
        file_content = response['Body'].read()
        df = pd.read_parquet(BytesIO(file_content))
        return df

In [11]:
bucket_name = 's3aravindh973515031797'
df = 'EV Charging Station/Charging_data.parquet'

In [12]:
df = read_parquet_from_s3(bucket_name, df)

In [13]:
df.head()

,Date,Time,EV Charging Demand (kW),Solar Energy Production (kW),Wind Energy Production (kW),Electricity Price ($/kWh),Grid Availability,Weather Conditions,Battery Storage (kWh),Charging Station Capacity (kW),...,Grid Stability Index,Carbon Emissions (kgCO2/kWh),Power Outages (hours),Energy Savings ($),Total Renewable Energy Production (kW),Effective Charging Capacity (kW),Adjusted Charging Demand (kW),Net Energy Cost ($),Carbon Footprint Reduction (kgCO2),Renewable Energy Efficiency
0,2021-01-01,00:00:00,0.112362,0.125388,0.009105,0.137310,Available,Partly Cloudy,16.532408,21.763422,...,0.731147,0.274944,1.889209,4.562581,0.134493,21.181550,0.028134,0.015428,0.023158,0.006350
1,2021-01-01,01:00:00,0.285214,0.052697,0.107589,0.125105,Available,Sunny,39.106930,31.215028,...,1.494387,0.481251,0.277371,0.215104,0.160286,27.639943,0.158721,0.035682,0.060875,0.005799
2,2021-01-01,02:00:00,0.219598,0.105035,0.043996,0.106661,Available,Cloudy,6.112691,46.489116,...,1.109293,0.146079,0.642644,0.029969,0.149031,41.781150,0.175614,0.023423,0.006425,0.003567
3,2021-01-01,03:00:00,0.179598,0.073839,0.275727,0.072209,Available,Partly Cloudy,30.041088,49.675029,...,0.847219,0.475255,0.546680,1.384950,0.349567,46.078097,0.004072,0.012969,0.083420,0.007586
4,2021-01-01,04:00:00,0.046806,0.068614,0.059824,0.091090,Available,Partly Cloudy,45.085422,21.166182,...,1.452466,0.319261,1.935850,1.872170,0.128438,17.733986,0.045315,0.004264,0.000476,0.007242
